# Import Libries

In [36]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics as mt

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
# from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score


# Load Dataset

In [37]:
# 1. Prepare os dados
df = pd.read_csv('train.csv')
df.head()

,id_cliente,idade,saldo_atual,divida_atual,renda_anual,valor_em_investimentos,taxa_utilizacao_credito,num_emprestimos,num_contas_bancarias,num_cartoes_credito,dias_atraso_dt_venc,num_pgtos_atrasados,num_consultas_credito,taxa_juros,investe_exterior,pessoa_polit_exp,limite_adicional
0,1767,21,278.172008,2577.05,24196.89636,104.306544,31.038763,6,5,7,21,14,9,15,Não,Não,Negar
1,11920,40,268.874152,2465.39,19227.37796,69.858778,36.917093,5,8,5,40,23,10,18,Não,Não,Negar
2,8910,36,446.643127,1055.29,42822.28223,134.201478,34.561714,0,3,6,26,13,3,15,Sim,Não,Negar
3,4964,58,321.141267,703.05,51786.82600,297.350067,31.493561,0,3,7,12,7,2,1,Sim,Não,Negar
4,10100,35,428.716114,891.29,44626.85346,134.201478,28.028887,2,8,7,24,10,8,20,Sim,Não,Negar


In [38]:
# Var resposta
df.loc[:,'limite_adicional'].unique()

array(['Negar', 'Conceder'], dtype=object)

In [39]:
# contagem de quantos negar/conceder
contagem = df.loc[:,'limite_adicional'].value_counts()
# Porcentagem da quantidade Negar/Conceder
porcentagem = df.loc[:,'limite_adicional'].value_counts(normalize=True)
print(contagem)
print(porcentagem)

Negar       7995
Conceder    1505
Name: limite_adicional, dtype: int64
Negar       0.841579
Conceder    0.158421
Name: limite_adicional, dtype: float64


In [40]:
# Conjunto de dados X (features) e y (rótulos/classes)
feactures = ['idade', 'saldo_atual', 'divida_atual', 'renda_anual',
       'valor_em_investimentos', 'taxa_utilizacao_credito', 'num_emprestimos',
       'num_contas_bancarias', 'num_cartoes_credito', 'dias_atraso_dt_venc',
       'num_pgtos_atrasados', 'num_consultas_credito', 'taxa_juros', 'investe_exterior', 'pessoa_polit_exp']
label = ['limite_adicional']      

In [41]:
# 2. Divida os dados em conjunto de treinamento
X_train = df.loc[:,feactures]
y_train = df.loc[:,label]

In [42]:
# Supondo que 'x_train' seja o DataFrame que contém seus dados de treinamento e devemos excluir dados nao relevantes ou que nao seja float/int
drop_n_relevante = ['investe_exterior', 'pessoa_polit_exp']
X_train = X_train.drop(drop_n_relevante, axis=1)

# Treinamento

In [43]:
# Aqui está um trecho de código que automatiza o treinamento do algoritmo K-NN 
# para encontrar o melhor valor para K, com base no exercício 2:

# 2. Defina uma lista de valores K para experimentar
k_values = [3, 5, 7, 9, 11, 13, 15, 17, 19, 21]

# 3. Crie uma instância do classificador KNN
knn = KNeighborsClassifier()

# 4. Defina os parâmetros para a busca por hiperparâmetros
param_grid = {'n_neighbors': k_values}

# # 5. Realize a busca por hiperparâmetros usando validação cruzada
# grid_search = GridSearchCV(knn, param_grid, cv=5) # cv é o número de folds na validação cruzada
# grid_search.fit(x_train, y_train.ravel())  # Usar y_train.ravel() em vez de y_train

# 5. Realize a busca por hiperparâmetros usando validação cruzada
grid_search = GridSearchCV(knn, param_grid, cv=7)
grid_search.fit(X_train, y_train.values.ravel())  # Converter y_train para um array numpy usando .values.ravel()


# 6. Obtenha os resultados da busca por hiperparâmetros
print("Melhores parâmetros: ", grid_search.best_params_)
print("Melhor acurácia média: ", grid_search.best_score_)

# Nesse código, a função GridSearchCV é utilizada para realizar uma busca exaustiva pelos melhores parâmetros. 
# O parâmetro param_grid especifica os valores que serão experimentados para o hiperparâmetro n_neighbors (valores de K). 
# O resultado da busca é armazenado em grid_search.best_params_, que contém os melhores parâmetros encontrados, e grid_search.best_score_, 
# que contém a melhor acurácia média obtida durante a validação cruzada.


Melhores parâmetros:  {'n_neighbors': 21}
Melhor acurácia média:  0.83789457102763


### Predicao

In [44]:
# 5. Faça previsões no conjunto de teste
y_pred =grid_search.predict(x_train)

In [45]:
# copy dataframe
df_result = df.copy()# veio todas
# criar uma col  classificacao
df_result['classificacao'] = y_pred

# 4.0 Performace

In [46]:
# Criar uma coluna Acertos com 1 e 0 , 1 para conceder e 0 para negar
df_result['acertos'] = df_result.loc[:,['id_cliente','limite_adicional','classificacao']].apply(lambda x: 1 if x['limite_adicional'] == x['classificacao'] else 0, axis=1)

In [47]:
# Verificando a col criada.
df_result.loc[:,['id_cliente','idade','limite_adicional','classificacao','acertos'] ].sample(10)

,id_cliente,idade,limite_adicional,classificacao,acertos
8910,8351,35,Negar,Negar,1
8969,9054,44,Negar,Negar,1
1222,9294,43,Negar,Negar,1
8752,11316,40,Negar,Negar,1
2308,2289,41,Negar,Negar,1
1713,10725,27,Negar,Negar,1
6514,6471,36,Negar,Negar,1
92,1262,28,Conceder,Negar,0
2259,11145,53,Negar,Negar,1
6937,8740,39,Negar,Negar,1


# Matriz da confusao

### Matriz da confusao

In [48]:
confusion = confusion_matrix(y_train, y_pred)
print("Matriz de Confusão:\n", confusion)

Matriz de Confusão:
 [[ 146 1359]
 [ 102 7893]]


### 4.1 Acuracia

In [49]:
# 6. Avalie o desempenho do modelo
accuracy = mt.accuracy_score(y_train, y_pred)
print("Acurácia: %.2f%%" % (accuracy * 100))

Acurácia: 84.62%


### 4.2 Precisao

In [51]:
precision = mt.precision_score(y_train, y_pred, pos_label='Conceder')
print("Precision:", precision)

Precision: 0.5887096774193549


### 4.3 Recall

In [50]:
recall = recall_score(y_train, y_pred, pos_label='Conceder')
print("Recall:", recall)

Recall: 0.09700996677740864
